In [19]:
import numpy as np
import time

def sphere (x) :
    return np.sum (np.square(x), axis=1)

def rastrigin (x) :
    A = 10
    return A*x.shape[1] + np.sum(np.square(x) - A*np.cos(2*np.pi*x) , axis=1)

def alpine (x) :
    return -np.prod (np.sqrt(x)*np.sin(x), axis=1)

# Inverse Parabolic Confined Distribution

In [20]:
def pso (obj, llim, rlim, Np=25, w=0.7, c1=2, c2=2, alpha=1.2, beta=0.9) :    
    def ipcd (particle, part, velocity) :
        nonlocal llim, rlim
        
        leftvio = part < llim
        rightvio = part > rlim
        leftRight = np.logical_or(part < llim, part > rlim)
        vio = np.sum (leftRight, axis=1).astype(bool)
        viosum = np.sum(vio)

        if viosum == 0 :
            return part, velocity

        leftvio = leftvio[vio]
        rightvio = rightvio[vio]
        partV = part[vio]
        particleV = particles[vio]

        limvec = np.copy(partV)
        limvec[leftvio] = np.tile (llim, (viosum, 1))[leftvio]
        limvec[rightvio] = np.tile (rlim, (viosum, 1))[rightvio]

        diff = partV - particleV
        Xnot = np.sqrt (np.sum (np.square(diff), axis=1, keepdims=True))
        kvec = np.min (np.where (diff == 0, 1, (limvec - particleV)/diff), axis=1, keepdims=True) 

        bvec = particleV + np.where (diff == 0, 0, kvec*diff)
        dvec = np.sqrt (np.sum (np.square(partV - bvec), axis=1, keepdims=True))

        Xpp = dvec + alpha*dvec*np.tan(np.random.rand(viosum).reshape(-1,1)*np.arctan((Xnot - dvec)/(alpha * dvec)))
        boundk = (Xnot - Xpp)/Xnot

        part[vio] = particleV + np.where (diff == 0, 0, boundk*diff)
        velocity[leftRight] *= -1
        return part, velocity
    
    D = len(llim)
    particles = np.array ([l + (r-l)*np.random.rand(Np) for l, r in zip(llim, rlim)]).transpose()
    velocity = np.random.rand(Np, D)
    momentum = np.zeros (shape=particles.shape)
    pbest = particles
    gbest = min (particles , key = lambda x : obj(x.reshape(1,-1))[0])

    for i in range (0, 1000) :
        r1 = np.random.rand (Np, D)
        r2 = np.random.rand (Np, D)

        momentum = beta*momentum + (1-beta)*velocity
        velocity = momentum + c1*r1*(pbest - particles) + c2*r2*(gbest - particles)

        particles, velocity = ipcd (particles, particles+velocity, velocity)  
        less = obj(particles) < obj(pbest)
        pbest[less] = particles[less]
        gbest = min (pbest , key = lambda x : obj(x.reshape(1,-1))[0])
    
    return gbest

In [21]:
llim = np.repeat(-10, 4)
rlim = np.repeat(10, 4)
lol = pso (sphere, llim, rlim)

print (lol)

[ 2.91097277e-47  3.39216556e-46  5.38245590e-47 -7.12685428e-47]
